<h1> Import dataset <h1>

In [1]:
# Download dataset and split it for training
import pandas as pd
import tabml.datasets
import numpy as np

df_dict = tabml.datasets.download_movielen_1m()
ratings, movies, users = df_dict["ratings"], df_dict["movies"], df_dict["users"]

<h1> Visualize data inclue: users, ratings, movies <h1>

In [2]:
# pass

<h1> Build simple SVD model <h1>

In [3]:
# Create the ratings matrix of shape (m×u) with rows as movies and columns as users
ratings_mat = np.ndarray(
    shape = (np.max(movies.MovieID),np.max(users.UserID)),
    dtype=np.uint8
)
ratings_mat[ratings.MovieID.values-1, ratings.UserID.values-1] = ratings.Rating.values

In [4]:
#Normalize
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, axis=1))]).T

In [5]:
# SVD matrix
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Calculate **cosine similarity**, sort by most similar and return the top N.

In [6]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Helper function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.MovieID == movie_id].Title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.MovieID == id].Title.values[0])

In [7]:
# Select k principal components to represent the movies, 
# a movie_id to find recommendations and print the top_n results.

k = 150
movie_id = 12 # Grab an id from movies.dat
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movies, movie_id, indexes)

Recommendations for Dracula: Dead and Loving It (1995): 

Dracula: Dead and Loving It (1995)
Tales from the Crypt Presents: Bordello of Blood (1996)
Puppet Master III: Toulon's Revenge (1991)
Return of the Texas Chainsaw Massacre, The (1994)
Children of the Corn IV: The Gathering (1996)
Children of the Corn III (1994)
My Boyfriend's Back (1993)
I Still Know What You Did Last Summer (1998)
Dead Man on Campus (1998)
Bio-Dome (1996)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21884\457065119.py:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
